In [2]:
import sys
from pathlib import Path
import os
os.environ['DATA_LOADER_CONFIG_PATH'] = str(Path.cwd().parent/"config"/"data_config.yaml")
extra_path = Path.cwd().parent
if str(extra_path) not in sys.path:
    sys.path.append(str(extra_path))
    print(f"Added: {extra_path}")

Added: /Users/guizhen/3_Water/GithubRepo/DataProcessToolBox


In [2]:
from data_loader import load_data

In [3]:
# Get the project root (parent of Notebook folder)
project_root = Path.cwd().parent

data = load_data('csv_data')
data

,id,name,department,salary,join_date
0,1,Alice Johnson,Engineering,95000,2022-01-15
1,2,Bob Smith,Marketing,72000,2021-11-03
2,3,Charlie Davis,Engineering,110000,2020-05-20
3,4,Diana Prince,Design,85000,2023-02-10
4,5,Evan Wright,Sales,64000,2022-08-30


In [3]:
from data_loader import get_loader
from data_loader.utils.file_finder import resolve_file_path
from pathlib import Path

# Get your loader
loader = get_loader()

# Your source name/pattern
source = "your_source_name"  # Replace with your actual source name

print("=== validate_source Debugging ===\n")

# Check what values validate_source will use
print("1. Loader's internal state:")
print(f"   loader.base_directory: {loader.base_directory}")
print(f"   loader.config_file_path: {loader.config_file_path}")
print(f"   loader.config: {loader.config}")

# 2. Manually replicate what validate_source does
print("\n2. Replicating validate_source logic:")
try:
    # This is what validate_source does internally
    resolved_path = resolve_file_path(
        source,
        base_directory=loader.base_directory,
        config_file_path=loader.config_file_path,
        match_strategy="first"
    )
    
    print(f"   ✓ resolve_file_path succeeded")
    print(f"   Resolved path: {resolved_path}")
    print(f"   Type: {type(resolved_path)}")
    
    # Handle list (from "all" strategy)
    if isinstance(resolved_path, list):
        if not resolved_path:
            print("   ✗ Empty list returned")
        else:
            file_path = resolved_path[0]
            print(f"   Using first from list: {file_path}")
    else:
        file_path = resolved_path
    
    # Check if it's a file
    print(f"\n3. File validation:")
    print(f"   file_path.exists(): {file_path.exists()}")
    print(f"   file_path.is_file(): {file_path.is_file()}")
    print(f"   file_path.is_dir(): {file_path.is_dir()}")
    print(f"   file_path: {file_path}")
    print(f"   file_path (absolute): {file_path.resolve()}")
    
    # Check readability
    print(f"\n4. Readability check:")
    try:
        with open(file_path, 'rb') as f:
            f.read(1)
        print("   ✓ File is readable")
    except Exception as e:
        print(f"   ✗ File read failed: {type(e).__name__}: {e}")
    
    # Now actually call validate_source
    print(f"\n5. Actual validate_source call:")
    result = loader.validate_source(source)
    print(f"   Result: {result}")
    
except Exception as e:
    print(f"   ✗ Error: {type(e).__name__}: {e}")
    import traceback
    traceback.print_exc()

print("\n=== End Debugging ===")

=== validate_source Debugging ===

1. Loader's internal state:
   loader.base_directory: ../TestData
   loader.config_file_path: /Users/guizhen/3_Water/GithubRepo/DataProcessToolBox/config/data_config.yaml
   loader.config: {'format_handler_config_path': None, 'base_directory': '../TestData', 'config_file_path': '/Users/guizhen/3_Water/GithubRepo/DataProcessToolBox/config/data_config.yaml'}

2. Replicating validate_source logic:
   ✗ Error: FileNotFoundError: No files found matching pattern 'your_source_name' in directory '/Users/guizhen/3_Water/GithubRepo/DataProcessToolBox/TestData'

=== End Debugging ===


Traceback (most recent call last):
  File "/var/folders/d5/phfhcf5s5b33hvmkd8xmzdlm0000gn/T/ipykernel_37006/563325766.py", line 23, in <module>
    resolved_path = resolve_file_path(
        source,
    ...<2 lines>...
        match_strategy="first"
    )
  File "/Users/guizhen/3_Water/GithubRepo/DataProcessToolBox/data_loader/utils/file_finder.py", line 296, in resolve_file_path
    return find_files_by_pattern(
        source,
    ...<2 lines>...
        match_strategy=match_strategy
    )
  File "/Users/guizhen/3_Water/GithubRepo/DataProcessToolBox/data_loader/utils/file_finder.py", line 202, in find_files_by_pattern
    raise FileNotFoundError(
    ...<2 lines>...
    )
FileNotFoundError: No files found matching pattern 'your_source_name' in directory '/Users/guizhen/3_Water/GithubRepo/DataProcessToolBox/TestData'


In [ ]:
from data_loader.utils.config_loader import load_data_config, get_data_config_path
from data_loader import get_loader

print("=== base_directory Debugging ===\n")

# 1. Check what's in the config file
print("1. Config file contents:")
try:
    config_file_path = get_data_config_path()
    print(f"   Config file: {config_file_path}")
    
    data_config = load_data_config()
    loader_config = data_config.get('loader', {}).get('config', {})
    print(f"   base_directory in config: {loader_config.get('base_directory')}")
    print(f"   Full loader config: {loader_config}")
except Exception as e:
    print(f"   ✗ Error loading config: {e}")

# 2. Check what the loader actually has
print("\n2. Loader instance:")
try:
    loader = get_loader()
    print(f"   loader.base_directory: {loader.base_directory}")
    print(f"   loader.config_file_path: {loader.config_file_path}")
    print(f"   loader.config: {loader.config}")
    print(f"   'base_directory' in loader.config: {'base_directory' in loader.config}")
except Exception as e:
    print(f"   ✗ Error: {e}")

# 3. Check if there's a path issue
print("\n3. Path resolution check:")
try:
    if loader.base_directory is None:
        print("   ⚠️  base_directory is None!")
        print("   This means path resolution will use config file directory or CWD")
        if loader.config_file_path:
            print(f"   Will use: {Path(loader.config_file_path).parent}")
        else:
            print(f"   Will use: {Path.cwd()}")
    else:
        print(f"   base_directory is set: {loader.base_directory}")
except Exception as e:
    print(f"   ✗ Error: {e}")

print("\n=== End Debugging ===")